In [1]:
import numpy as np  # For array manipulation and mathematical operations
import pandas as pd  # For working with datasets
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns  # For enhanced data visualization
import scipy.stats  # For statistical functions

In [2]:
t_range = [4,12] #range of thickness in mm 
angle_range = [110,179] #range of angles in degrees
AR_range = [0.01,0.7] #range of edge radii AR in mm
height_range = [80,120]
k_ideal = 330 # Ideal Stiffness in N/mm 
mutation_prob = 0.05
des_variables = ['Thickness','Angle','AR','Height']
res_variables = ['delh','volume','k','fitness']
head = des_variables+res_variables
pop_num = 12

In [38]:
def generate_initial_designs(a, b, c, d, pop):
    designs_initial = []
    for _ in range(pop):  # Generate 12 arrays
        x = np.random.triangular(a[0], (a[0] + a[1]) / 2, a[1])  # Random thickness
        y = np.random.triangular(b[0], (b[0] + b[1]) / 2, b[1])  # Random angle
        z = np.random.triangular(c[0], (c[0] + c[1]) / 2, c[1])  # Random edge radii AR
        w = np.random.triangular(d[0], (d[0] + d[1]) / 2, d[1])  # Random thickness

        x = round(x, 2)
        y = round(y, 2)
        z = round(z, 2)
        w = round(w, 2)

        # Empty values for new variables
        delh = None
        volume = None
        k = None
        fitness = None

        designs_initial.append([x, y, z, w, delh, volume, k, fitness])  # Append the array


    return designs_initial

def cost_function(force,del_h,v,a,b,k_ideal,v_ideal):
    k = force/del_h # k in [N/mm]
    f = (a*((k_ideal-k)^2)/(k_ideal^2)+b*(v/v_ideal))^-1
    return f 

def mutate(df,mutation_prob,t_range,angle_range,AR_range,height_range):
    for index, row in df.iterrows():
        for column in df.columns:
            # Generate a random number between 0 and 1
            rand_prob = np.random.rand()
            if rand_prob < mutation_prob:
                if column == 'thickness':
                    new_value = np.random.uniform(t_range[0], t_range[1])
                elif column == 'angle':
                    new_value = np.random.uniform(angle_range[0], angle_range[1])
                elif column == 'AR':
                    new_value = np.random.uniform(AR_range[0], AR_range[1])
                elif column == 'height':
                    new_value = np.random.uniform(height_range[0], height_range[1])
                else: 
                    new_value = df.at[index, column]
                
                # Update the cell value
                df.at[index, column] = new_value
    return df

def genetic_algorithm(df,t_range,angle_range,AR_range,height_range,mutation_prob):
    # Reindex the DataFrame from 0
    df.reset_index(drop=True, inplace=True)

    # Add all fitness variables to create a fitness sum
    fitness_sum = df['fitness'].sum()

    # Calculate relative fitness for each row
    df['relative_fitness'] = df['fitness'] / fitness_sum
    print(df)

    # Use relative fitness as probabilities for selection
    selected_indices = np.random.choice(df.index, size=8, replace=False, p=df['relative_fitness'])
    print(selected_indices)
    best_design_index = df['fitness'].idxmax()
    best_design = df.loc[best_design_index]
    keeper = [best_design['Thickness'],best_design['Angle'],best_design['AR'],best_design['Height']]
    print(keeper)

    # Select 8 designs based on their probability of selection
    selected_designs = df.loc[selected_indices]
    print(selected_designs)

    # Mate selected rows with a single point mutation
    children = []
    parent1 = []
    parent2 = []
    selected_pairs = set()  # Set to store selected pairs
    while len(children) < 11:
        # Randomly select a pair of indices
        pair_indices = tuple(sorted(np.random.choice(selected_designs.index, size=2, replace=False)))
        # Check if the pair is already selected
        if pair_indices not in selected_pairs:
            # Add the pair to the set of selected pairs
            selected_pairs.add(pair_indices)
            # Take Parents
            parent1 = df.loc[pair_indices[0]]
            parent2 = df.loc[pair_indices[1]]
            # Randomly decide which parent contributes which variable
            if np.random.choice([True, False]):  # 50% chance for swap
                child = [parent1['Thickness'], parent1['Angle'],parent2['AR'],parent2['Height']]
            else:
                child = [parent2['Thickness'], parent2['Angle'],parent1['AR'],parent1['Height']]
            # Append child to children list
            children.append(child)
    print(children)
    children = pd.DataFrame(children, columns=['Thickness','Angle','AR','Height'])
    print(children)
    children = mutate(children,mutation_prob,t_range,angle_range,AR_range,height_range)
    print(children)
    keeper_df = pd.DataFrame([keeper], columns=children.columns)
    # Append the new row to the existing DataFrame
    children = children.append(keeper_df, ignore_index=True)

    return children


The following cell was only run in order to generate an initial population of 12 designs, with variables randomly spread throughout the respective design variable ranges.

init_design = generate_initial_designs(t_range,angle_range,AR_range,height_range,pop_num) 
df = pd.DataFrame(init_design, columns=head, index=range(1, len(init_design) + 1))
print(df)
df.to_csv('parents.csv', index_label='Index')

The following cell reads in a design data frame from a CSV file. It is assumed that this data frame is organized as a [7,12] package. 
The headings for each of the columns should be 'Index' 'Thickness' 'Angle' 'Radii' 'delh' 'volume' 'k' 'fitness'.
The genetic algorithm starts by adding a new column 'relative fitness' which is calculated by dividing each designs fitness by the sum of all fitnesses.
This relative fitness is used as probability of selection for the designs as parents. 8 parents are selected to a mating pool, and 24 unique pairs are selected. 
Each of these pairs produces one child with a single point crossover. 
Following the generation of the initial children, the genes are mutated within their respecitve ranges, and at a frequency dictated by the 'mutation_prob' parameter. 

In [39]:
new_df = pd.read_csv('parents.csv', index_col='Index')
print(new_df)
children = genetic_algorithm(new_df,t_range,angle_range,AR_range,height_range,mutation_prob)




       Thickness   Angle    AR  Height   delh  total vol    volume          k  \
Index                                                                           
1           6.10  141.62  0.16  112.71   9.52   0.016689  0.004026   598.8024   
2           4.00  120.00  0.53  102.08  26.73   0.015243  0.003296   213.2435   
3           4.65  112.26  0.53  102.08  25.33   0.015243  0.003578   225.0296   
4           4.00  120.00  0.29  112.71  27.10   0.016689  0.003549   210.3321   
5           9.16  139.25  0.53  102.08   5.31   0.015243  0.004448  1073.4463   
6           6.10  141.62  0.29  112.71   9.26   0.016689  0.003898   615.4178   
7           5.53  124.45  0.67   82.74   7.79   0.012613  0.003353   732.0832   
8           9.16  139.25  0.67   82.74   3.33   0.012613  0.003966  1711.7117   
9           6.00  165.52  0.53  102.08   5.38   0.015243  0.003472  1059.2827   
10          6.00  165.52  0.67   82.74   3.43   0.012613  0.003181  1664.2336   
11          9.16  139.25  0.

C:\Users\Cameron\AppData\Local\Temp\ipykernel_22536\1620256825.py:101: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  children = pd.concat(children,keeper)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

Procedure for testing

1) Run initial design generation and paste designs into the sheet on the drive.

2) Make one of the designs in Catia by tweaking parameters.

3) Save Catia part and upload to Abacus for simulation.

4) Remesh part for simulation.

5) Run simulation.

6) Take the average change in height, and the volume of the part.

7) Repeat steps 2-6 until all delH and V's are recorded.

8) Enter delH and V's into corresponding generation sheet on drive, sheet will automatically populate with 'k' and 'fitness'.

9) Coppy and paste full dataframe from sheet into local csv file.

10) Run genetic code using local CSV.

11) The code will output the childen designs onto the second sheet locally. 

12) Copy and paste the local children data frame into next generation sheet on drive. 

13) Repeat steps 2-12 until the end of the optimization. 